# Integrated Project
***
In this notebook, We'll be working for the online game store Ice, looking at historical data on games in order to make a few predictions about the coming year's trends. 

The data was collected in 2016 and contains "User and expert reviews, genres, platforms (e.g. Xbox or PlayStation), and historical data on game sales..." which we will look at, clean if necessary, then perform some statistical analysis to answer a few questions about the data.

**Hypotheses:** 

We will test the following hypotheses:

- Average user ratings of the Xbox One and PC platforms are the same. 

- Average user ratings for the Action and Sports genres are different.

The alpha value will be 0.01% to ensure accurate results.

In [1]:
# Importing all necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats as st

In [2]:
# Formating Markdown tables
from IPython.core.display import HTML
table_css = 'table {align:left;display:block} '
HTML('<style>{}</style>'.format(table_css))

## Data Description
***

In [3]:
# Importing the data into a pandas DataFrame
try:
    df = pd.read_csv('games.csv')
except Exception as e:
    print(f'Unexpected {e=}, {type(e)=}')
else:
    df = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/moved_games.csv')

# Reading a brief description and sample of the data
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


There are 16715 rows and 11 columns in the DataFrame.

**From the Documentation:**

- `Name` : Name of the game

- `Platform` : The system the game was released on

- `Year_of_Release` : The year the game was released

- `Genre` : The genre of the game (e.g. Action, Platform)

- `NA_sales` : Total sales in North America (in USD milion)

- `EU_sales` : Total sales in Europe (in USD milion)

- `JP_sales` : Total sales in Japan (in USD milion)

- `Other_sales` : Total sales in other regions (in USD milion)

- `Critic_Score` : Average Critic Rating (Maximum 100)

- `User_Score` : Average User Rating (Maximum 10)

- `Rating` : The rating given to the game by the ESRB

Data for 2016 may be incomplete.


In [4]:
# Generating descriptive statistics of the data for preliminary analysis
display(df.describe(include='object'))
df.describe()

,Name,Platform,Genre,User_Score,Rating
count,16713,16715,16713,10014,9949
unique,11559,31,12,96,8
top,Need for Speed: Most Wanted,PS2,Action,tbd,E
freq,12,2161,3369,2424,3990


,Year_of_Release,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score
count,16446.000000,16715.000000,16715.000000,16715.000000,16715.000000,8137.000000
mean,2006.484616,0.263377,0.145060,0.077617,0.047342,68.967679
std,5.877050,0.813604,0.503339,0.308853,0.186731,13.938165
min,1980.000000,0.000000,0.000000,0.000000,0.000000,13.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,60.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,71.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,79.000000
max,2016.000000,41.360000,28.960000,10.220000,10.570000,98.000000


**Issues To address:**

1. There are missing values in the `Name`, `Year_of_Release`, `Genre`, `Critic_Score`, `User_Score`, and `Rating` columns.

2. `Year_of_Release` will be converted to `int` datatype.

3. *Need for Speed: Most Wanted* appears in the data 12 times, will check for duplicate entries.

4. `User_Score` will need to be converted to int, and *tbd* entries will be addressed.

## Data Cleaning
***

First, column names will be corrected and the `total_sales` column will be created.

Then the data will be checked for duplicate values, which would then be excluded, then the missing values will be either filled or excluded based on what can be infered from the data.

In [5]:
# Correcting column names
df.columns = df.columns.str.lower()

# Adding total_sales column
df.insert(8, 'total_sales', df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   total_sales      16715 non-null  float64
 9   critic_score     8137 non-null   float64
 10  user_score       10014 non-null  object 
 11  rating           9949 non-null   object 
dtypes: float64(7), object(5)
memory usage: 1.5+ MB


### Duplicates

In [6]:
# Checking for duplicates
check = ['name', 'platform']
display(f'Duplicates: {df.duplicated().sum()}')
display(f'Filtered Duplicates: {df[check].duplicated().sum()}')

# Looking at duplicate data
dupes = pd.DataFrame()
dupes =  df[df[check].duplicated(keep=False)]
display(dupes.sort_values(by='name'))

'Duplicates: 0'

'Filtered Duplicates: 5'

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,total_sales,critic_score,user_score,rating
604,Madden NFL 13,PS3,2012.0,Sports,2.11,0.22,0.00,0.23,2.56,83.0,5.5,E
16230,Madden NFL 13,PS3,2012.0,Sports,0.00,0.01,0.00,0.00,0.01,83.0,5.5,E
1190,Need for Speed: Most Wanted,X360,2012.0,Racing,0.62,0.78,0.01,0.15,1.56,83.0,8.5,T
1591,Need for Speed: Most Wanted,X360,2005.0,Racing,1.00,0.13,0.02,0.10,1.25,83.0,8.5,T
5972,Need for Speed: Most Wanted,PC,2005.0,Racing,0.02,0.23,0.00,0.04,0.29,82.0,8.5,T
11715,Need for Speed: Most Wanted,PC,2012.0,Racing,0.00,0.06,0.00,0.02,0.08,82.0,8.5,T
1745,Sonic the Hedgehog,PS3,2006.0,Platform,0.41,0.06,0.04,0.66,1.17,43.0,4.1,E10+
4127,Sonic the Hedgehog,PS3,NaN,Platform,0.00,0.48,0.00,0.00,0.48,43.0,4.1,E10+
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,2.39,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,0.03,NaN,NaN,NaN


**Steps to Take:**

- Duplicate *Madden NFL 13* entry will be removed.

- The duplicate *Sonic the Hedgehog* entry is missing  the `year_of_release` cell. Because the second entry contains significant data in the `eu_sales` column we will combine that data with the original before removal.

- *Need for Speed: Most Wanted* entries will be left alone as the `year_of_release` column tells us they are different games.

- The *NaN* entry will be removed from our dataset as there is not enough information to determine what game it was.

In [7]:
# Addressing Sonic the Hedgehog
df.at[1745, 'eu_sales'] = df.at[1745, 'eu_sales'] + df.at[4127, 'eu_sales']
df.at[1745, 'total_sales'] = df.at[1745, 'total_sales'] + df.at[4127, 'total_sales']
print(df.loc[1745])
df.at[4127, 'year_of_release'] = df.at[1745, 'year_of_release']

# Removing duplicate entries
check.append('year_of_release')
df = df.drop_duplicates(check, keep='first').reset_index(drop=True)

# Checking for duplicates
display(f'Duplicated Items: {df[check].duplicated().sum()}')
df.info()

name               Sonic the Hedgehog
platform                          PS3
year_of_release                2006.0
genre                        Platform
na_sales                         0.41
eu_sales                         0.54
jp_sales                         0.04
other_sales                      0.66
total_sales                      1.65
critic_score                     43.0
user_score                        4.1
rating                           E10+
Name: 1745, dtype: object


'Duplicated Items: 0'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16712 entries, 0 to 16711
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16711 non-null  object 
 1   platform         16712 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16711 non-null  object 
 4   na_sales         16712 non-null  float64
 5   eu_sales         16712 non-null  float64
 6   jp_sales         16712 non-null  float64
 7   other_sales      16712 non-null  float64
 8   total_sales      16712 non-null  float64
 9   critic_score     8135 non-null   float64
 10  user_score       10012 non-null  object 
 11  rating           9947 non-null   object 
dtypes: float64(7), object(5)
memory usage: 1.5+ MB


3 duplicate entries were removed from the dataset.

### Missing Values

In [8]:
# Removing game with missing name
df = df.dropna(subset=['name','genre'])
df.isna().sum()

name                  0
platform              0
year_of_release     268
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
total_sales           0
critic_score       8576
user_score         6699
rating             6764
dtype: int64

#### `year_of_release`

In [9]:
# Calculating percentage of missing entries
yor_per = df['year_of_release'].isna().sum() / df['name'].count()
display(f'Percent of missing entries in the year_of_release column: {yor_per:.2%}')

# Filling missing values and converting datatype
df['year_of_release'] = df['year_of_release'].fillna(0).astype('int')
df.info()

'Percent of missing entries in the year_of_release column: 1.60%'

<class 'pandas.core.frame.DataFrame'>
Index: 16711 entries, 0 to 16711
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16711 non-null  object 
 1   platform         16711 non-null  object 
 2   year_of_release  16711 non-null  int32  
 3   genre            16711 non-null  object 
 4   na_sales         16711 non-null  float64
 5   eu_sales         16711 non-null  float64
 6   jp_sales         16711 non-null  float64
 7   other_sales      16711 non-null  float64
 8   total_sales      16711 non-null  float64
 9   critic_score     8135 non-null   float64
 10  user_score       10012 non-null  object 
 11  rating           9947 non-null   object 
dtypes: float64(6), int32(1), object(5)
memory usage: 1.6+ MB


Only 1.6% of the `year_of_release` column is missing data. This is an acceptable amount and will not be filled in to maintain the integrity of the data.

Instead, the data will be marked with a default placeholder(0) and filtered during analysis, so the column can be changed to the `int` datatype, to ensure all entries are whole years.

#### `rating`



Missing values will be filled with the string 'unknown'. Since these columns primarily serve to categorize data, introducing an default variable for the column will minimally disrupt the dataset. The `rating` column can then potentially be used to generate a more contextualized average for other datasets.

In [10]:
# Replacing missing values with 'unknown'
df['rating'] = df['rating'].fillna('unknown')
df.isna().sum()

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
total_sales           0
critic_score       8576
user_score         6699
rating                0
dtype: int64

#### `user_score`

In [11]:
# Calculating percentage of missing entries
user_per = df['user_score'].isna().sum() / df['name'].count()
f'Percent of missing entries in the user_score column: {user_per:.2%}'

'Percent of missing entries in the user_score column: 40.09%'

40.09% of values missing from this column. However as before, this column contains placeholder *tbd* entries that indicate `user_score` was unavailable at the time of data collection, likely because it was still being calculated. 

In [12]:
# Displaying total 'tbd' entries
df['user_score'].value_counts().head(1)

user_score
tbd    2424
Name: count, dtype: int64

All entries with the *tbd* value will be converted into NaN and then the column will be coverted to the `float64` datatype as the column should have been numerical.

In [13]:
# Replacing 'tbd'
df['user_score'].replace('tbd', np.nan, inplace=True)
user_per = df['user_score'].isna().sum() / df['name'].count()
display(f'Percent of missing entries in the rating column: {user_per:.2%}')

# Converting datatype
df['user_score'] = pd.to_numeric(df['user_score'])
df.info()

C:\Users\m-gle\AppData\Local\Temp\ipykernel_34320\1319855985.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['user_score'].replace('tbd', np.nan, inplace=True)


'Percent of missing entries in the rating column: 54.59%'

<class 'pandas.core.frame.DataFrame'>
Index: 16711 entries, 0 to 16711
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16711 non-null  object 
 1   platform         16711 non-null  object 
 2   year_of_release  16711 non-null  int32  
 3   genre            16711 non-null  object 
 4   na_sales         16711 non-null  float64
 5   eu_sales         16711 non-null  float64
 6   jp_sales         16711 non-null  float64
 7   other_sales      16711 non-null  float64
 8   total_sales      16711 non-null  float64
 9   critic_score     8135 non-null   float64
 10  user_score       7588 non-null   float64
 11  rating           16711 non-null  object 
dtypes: float64(7), int32(1), object(4)
memory usage: 1.6+ MB


A formula that takes the average score for games released on multiple consoles will be used to fill in gaps in the missing data.

In [14]:
# Creating a function 
def fill_avg_score(group):
    return group.fillna(group.mean())

# Applying the function
df['user_score'] = df.groupby('name')['user_score'].transform(fill_avg_score)
user_per = df['user_score'].isna().sum() / df['name'].count()
f'Percent of missing entries in the rating column: {user_per:.2%}'

'Percent of missing entries in the rating column: 48.02%'

48.02% of the data is still missing after applying the function.

In [15]:
# Generating statistics of data missing user_score
df[df['user_score'].isna()].describe()

,year_of_release,na_sales,eu_sales,jp_sales,other_sales,total_sales,critic_score,user_score
count,8024.000000,8024.000000,8024.000000,8024.000000,8024.000000,8024.000000,666.000000,0.0
mean,1976.779287,0.168263,0.076402,0.101042,0.019773,0.365480,59.966967,NaN
std,237.426948,0.728224,0.315482,0.349070,0.083857,1.238788,11.689033,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.000000,NaN
25%,2000.000000,0.000000,0.000000,0.000000,0.000000,0.040000,53.000000,NaN
50%,2007.000000,0.020000,0.000000,0.000000,0.000000,0.110000,61.000000,NaN
75%,2010.000000,0.120000,0.040000,0.070000,0.010000,0.290000,68.000000,NaN
max,2016.000000,29.080000,10.950000,10.220000,2.740000,40.240000,89.000000,NaN


While there are some obvious outliers, most games missing `user_score` data did not sell well and were reviewed poorly by critics. The missing data will be excluded from analysis.

#### `critic score`

In [16]:
# Calculating percentage of missing entries
critic_per = df['critic_score'].isna().sum() / df['name'].count()
f'Percent of missing entries in the year_of_release column: {critic_per:.2%}'

'Percent of missing entries in the year_of_release column: 51.32%'

51.32% of values are missing from this column! Over half of the dataset, which can be assumed to not have been avalible at the time of data collection, which could be due to a variety of factors including being calculated or due to the age of the game.

The `fill_avg_score` function defined earlier will be used to fill in spots of the data.

In [17]:
# Applying the formula
df['critic_score'] = df.groupby('name')['critic_score'].transform(fill_avg_score)
critic_per = df['critic_score'].isna().sum() / df['name'].count()
f'Percent of missing entries in the year_of_release column: {critic_per:.2%}'

'Percent of missing entries in the year_of_release column: 45.57%'

45.57% of the data is still missing. The remaining data will be left empty as we cannot reasonably fill in such a large portion of data. 

On average, as with `user_score`, the games missing the `critic_score` column either did not sell well, weren't reviewed well by users, and/or are older games:

In [18]:
# Generating statistics of data missing critic_score
df[df['critic_score'].isna()].describe()

,year_of_release,na_sales,eu_sales,jp_sales,other_sales,total_sales,critic_score,user_score
count,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,0.0,336.000000
mean,1979.340250,0.174014,0.079132,0.105858,0.020506,0.379510,NaN,6.484524
std,227.290314,0.747673,0.324453,0.357386,0.085844,1.270902,NaN,2.060097
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000
25%,1999.000000,0.000000,0.000000,0.000000,0.000000,0.040000,NaN,5.275000
50%,2007.000000,0.010000,0.000000,0.010000,0.000000,0.110000,NaN,7.000000
75%,2010.000000,0.120000,0.040000,0.070000,0.010000,0.300000,NaN,8.000000
max,2016.000000,29.080000,10.950000,10.220000,2.740000,40.240000,NaN,9.700000


The data has been sufficiently cleaned and enhanced for the purposes of making the projection.

At this point:
- Column names were cleaned and `total_sales` colum was created.
- Duplicate entries have been removed
- Missing values were filled with appropriate placeholders or left empty to preserve the integrity of the data.

## Data Analysis
***

The data will now be grouped and visualized in order to statistically analyse it and answer a few questions.

### Filtering Data

1. Look at how many games were released in different years. Is the data for every period significant? Determine what period you should take data for.

In [19]:
#Filtering placeholder values
df_filtered = df[df['year_of_release'] != 0]

# Displaying a histogram of release year
release_year = px.histogram(df_filtered, x='year_of_release', title='Histogram of Release Years')
release_year.update_layout(xaxis_title='Year of Release', yaxis_title='Total Number of Games')

The data is primarily centered around the late 2000's era with outliers in the 1980's and 1990's. The older data is likely not relevant to our study and will be excluded, as the goal is to model 2017.

2. Look at how sales varied from platform to platform. Choose the platforms with the greatest total sales and build a distribution based on data for each year. Find platforms that used to be popular but now have zero sales. How long does it generally take for new platforms to appear and old ones to fade?  Which platforms are leading in sales? Which ones are growing or shrinking? Select several potentially profitable platforms.

In [20]:
# Looking at top preforming consoles and filtering the dataset
total_sales = df.groupby('platform')['total_sales'].sum().sort_values(ascending=False)
total_sales_50m = total_sales[total_sales > 50]
display(total_sales_50m, f'Platforms with Total Revenue over $50m: {len(total_sales_50m)}')

# Applying the filter
df_filtered = df_filtered[df_filtered['platform'].isin(total_sales_50m.index)]

platform
PS2     1255.77
X360     971.42
PS3      939.64
Wii      907.51
DS       806.12
PS       730.86
GBA      317.85
PS4      314.14
PSP      294.05
PC       259.52
3DS      259.00
XB       257.74
GB       255.46
NES      251.05
N64      218.68
SNES     200.04
GC       198.93
XOne     159.32
2600      96.98
WiiU      82.19
PSV       54.07
Name: total_sales, dtype: float64

'Platforms with Total Revenue over $50m: 21'

In [21]:
# Grouping data by platform and year_of_release
platform = df_filtered.groupby(['platform','year_of_release'])['total_sales'].sum().reset_index()

# Creating the figure
sales_timeline = px.line(platform,
                         x='year_of_release',
                         y='total_sales',
                         color='platform',
                         symbol='platform',
                         title='Timeline of Annual Sales')

# Adding labels
sales_timeline.update_layout(xaxis_title='Year of Release',
                             yaxis_title='Annual Sales')

sales_timeline.show()

Generally consoles peak in performance within the first few years of it's release and can stay in that range anywhere from 1 to 4 years, then sales tend to decline for about 5 years until the end of that console's lifespan. New consoles tend to be released one or two years before the end the previous console's lifespan, leading to a new console approximately every decade.

**Platforms leading in all time sales:**
- *PS2*   -- $1255.77 m

- *XBox 360* --  $971.42 m

- *PS3*  --  $939.64 m

- *Wii*  --  $907.51 m

- *DS*   --  $806.12 m

- *PS*   --  $730.86 m

- *GBA*  --  $317.85 m

- *PS4*  --  $314.14 m

**Growing Platforms:**
- *PS4*
- *XBox One*

**Shrinking Platforms:**
- *XBox 360*
- *PS3*
- *PSP*
- *WiiU*
- *3DS*

To make a more accurate preditiction, the data will be restricted to 2013, the year the *PS4* and *XBox One* were released, as they are the current growing market. While the *3DS* is losing value it still holds a high market share and will be included in futher analysis.

3. Build a box plot for the global sales of all games, broken down by platform. Are the differences in sales significant? What about average sales on various platforms?

Values over $5 m are likely console exclusive titles that have been excluded from the graph for better visualization.

In [22]:
# Reviewer's code 2

df.query('year_of_release>2011').pivot_table(index='year_of_release', columns='platform', values='total_sales', aggfunc='sum')

platform,3DS,DS,PC,PS3,PS4,PSP,PSV,Wii,WiiU,X360,XOne
year_of_release,,,,,,,,,,,
2012,51.36,11.01,23.22,107.35,NaN,7.69,16.19,21.71,17.56,99.74,NaN
2013,56.57,1.54,12.38,113.25,25.99,3.14,10.59,8.59,21.65,88.58,18.96
2014,43.76,NaN,13.28,47.76,100.00,0.24,11.90,3.75,22.03,34.74,54.07
2015,27.78,NaN,8.52,16.82,118.90,0.12,6.25,1.14,16.35,11.96,60.14
2016,15.14,NaN,5.25,3.60,69.25,NaN,4.25,0.18,4.60,1.52,26.15


In [23]:
# Filtering data
df_filtered = df_filtered[df_filtered['year_of_release'] > 2012]

# Graphing
sales_box = px.box(df_filtered,
                   x='platform',
                   y='total_sales',
                   color='platform',
                   title='Boxplot of Total Sales',
                   range_y=[-0.25,5])

# Adding labels
sales_box.update_layout(xaxis_title='Platform',
                        yaxis_title='Total Sales')

sales_box.show()

Over the selected period, the *PS3*, *XBox 360*, *PS4*, and *XBox One* all have similar distributions and most of the consoles have an average `total_sales` of approximately $0.2 m with the exceptions of *3DS*, *PC*, *PSV*, and *PSP*. 

4. Take a look at how user and professional reviews affect sales for one popular platform (PS4). Build a scatter plot and calculate the correlation between reviews and sales.

In [24]:
# Selecting PS4 games
df_ps4 = df_filtered[df_filtered['platform'] == 'PS4']
len(df_ps4)

392

In [25]:
# Creating scatterplots for 'user_score' and 'critic_score'
ps4_critic = px.scatter(df_ps4, x= 'critic_score', y='total_sales', title='PS4 Critic Score Distribution')
ps4_user = px.scatter(df_ps4, x='user_score', y='total_sales', title='PS4 User Score Distribution')

# Adding labels
ps4_critic.update_layout(xaxis_title='Critic Score', yaxis_title='Total Sales')
ps4_user.update_layout(xaxis_title='User Score', yaxis_title='Total Sales')

# Generating correlation coefficient
ps4_critic_corr = df_ps4['total_sales'].corr(df_ps4['critic_score'])
ps4_user_corr = df_ps4['total_sales'].corr(df_ps4['user_score'])

# Displaying graphs and correlation coefficient
ps4_critic.show()
display(f'Correlation coefficient between "critic_score" and "total_sales": {ps4_critic_corr}')
ps4_user.show()
display(f'Correlation coefficient between "user_score" and "total_sales": {ps4_user_corr}')

'Correlation coefficient between "critic_score" and "total_sales": 0.39155120045082015'

'Correlation coefficient between "user_score" and "total_sales": -0.021612493200478526'

There appears to be a somewhat positive correlation between `critic_score` and `total_sales`, whereas there seems to be little to no relation between `user_score` and `total_sales` for *PS4* games.

5. Compare the sales of the same games on other platforms.

In [26]:
# Selecting XOne games
df_xone = df_filtered[df_filtered['platform'] == 'XOne']
len(df_xone)

247

In [27]:
# Creating scatterplots for 'user_score' and 'critic_score'
xone_critic = px.scatter(df_xone, x= 'critic_score', y='total_sales', title='XBox One Critic Score Distribution')
xone_user = px.scatter(df_xone, x='user_score', y='total_sales', title='XBox One User Score Distribution')

# Adding labels
xone_critic.update_layout(xaxis_title='Critic Score', yaxis_title='Total Sales')
xone_user.update_layout(xaxis_title='User Score', yaxis_title='Total Sales')

# Generating correlation coefficient
xone_critic_corr = df_xone['total_sales'].corr(df_xone['critic_score'])
xone_user_corr = df_xone['total_sales'].corr(df_xone['user_score'])

# Displaying graphs and correlation coefficient
xone_critic.show()
display(f'Correlation coefficient between "critic_score" and "total_sales": {xone_critic_corr}')
xone_user.show()
display(f'Correlation coefficient between "user_score" and "total_sales": {xone_user_corr}')

'Correlation coefficient between "critic_score" and "total_sales": 0.37398972351240567'

'Correlation coefficient between "user_score" and "total_sales": -0.09402749243006256'

In [28]:
# Selecting 3DS games
df_3ds = df_filtered[df_filtered['platform'] == '3DS']
len(df_3ds)

303

In [29]:
# Creating scatterplots for 'user_score' and 'critic_score'
ds_critic = px.scatter(df_3ds, x= 'critic_score', y='total_sales', title='3DS Critic Score Distribution')
ds_user = px.scatter(df_3ds, x='user_score', y='total_sales', title='3DS User Score Distribution')

# Adding labels
ds_critic.update_layout(xaxis_title='Critic Score', yaxis_title='Total Sales')
ds_user.update_layout(xaxis_title='User Score', yaxis_title='Total Sales')

# Generating correlation coefficient
ds_critic_corr = df_3ds['total_sales'].corr(df_3ds['critic_score'])
ds_user_corr = df_3ds['total_sales'].corr(df_3ds['user_score'])

# Displaying graphs and correlation coefficient
ds_critic.show()
display(f'Correlation coefficient between "critic_score" and "total_sales": {ds_critic_corr}')
ds_user.show()
display(f'Correlation coefficient between "user_score" and "total_sales": {ds_user_corr}')

'Correlation coefficient between "critic_score" and "total_sales": 0.3431975838675125'

'Correlation coefficient between "user_score" and "total_sales": 0.2873122626296861'

The *XBox One* platform has similar distributions to the *PS4* platform, and the correlation coefficients for the graphs reflect that. The *3DS* platform has a similar correlation coefficient with critic score, however the `user_score` graph has a much more posititve relationship than the other two platforms. That can be attributed to lower total sales overall for the *3DS*.

6. Take a look at the general distribution of games by genre. What can we say about the most profitable genres? Can you generalize about genres with high and low sales?

In [30]:
# Grouping by genre
genre = df_filtered.groupby('genre')['total_sales'].mean().sort_values()

# Graphing
genre_graph = px.bar(genre, x='total_sales', title='Average Genre Sales')
genre_graph.update_layout(xaxis_title='Average Sales', yaxis_title='Genre')
genre_graph.show()

The average top-selling game genres include *Shooter*, *Sports*, *Platform*, and *Role-Playing*. *Shooter* games stand out as particularly popular, while genres requiring critical thinking such as *Puzzle*, *Strategy* and *Simulation* are comparatively less favored. This preference may stem from the widespread appeal of action-packed experiences over those that challenge cognitive skills. For instance, the enduring popularity of *Sports* games could be attributed to the widespread passion for sports teams, both in playing and supporting them.

## Regional Profiles
***
**For each region (NA, EU, JP), determine:**

- The top five platforms. Describe variations in their market shares from region to region.

- The top five genres. Explain the difference.

- Do ESRB ratings affect sales in individual regions?

### Platform

In [31]:
# Grouping by platform
df_platform = df_filtered.groupby('platform')[['na_sales', 'eu_sales', 'jp_sales']].sum().sort_values(by=['na_sales'])

# Graphing
platform_bar = px.bar(df_platform, barmode='group', title='Platform Sales by Region')
platform_bar.update_layout(xaxis_title='Platform', yaxis_title='Total Sales')
platform_bar.show()

**Top Platforms for each Region:**

| | NA   | EU   | JP   |
|-| :--- | :--- | :--- |
|1| PS4  | PS4  | 3DS  |
|2| XOne | PS3  | PS3  |
|3| X360 | XOne | PSV  |
|4| PS3  | X360 | PS4  |
|5| 3DS  | 3DS  | WiiU |


- The *PS4*, *PS3*, and *3DS* are all in the top 5 for each region.
- The *PS4* is extremely dominant in the EU market, and leading in the NA market.
- *XBox One* and *XBox 360* perform well in NA and EU markets but are almost nonexistent in the JP market.
- *3DS* leads the JP market by a large margin but is 5th in both other markets. Nintendo has strong sway in this region, followed by Sony.

### Genre

In [32]:
# Grouping by genre
df_genre = df_filtered.groupby('genre')[['na_sales', 'eu_sales', 'jp_sales']].sum().sort_values(by='na_sales')

# Graphing
genre_bar = px.bar(df_genre, barmode='group', title='Genre Sales by Region')
genre_bar.update_layout(xaxis_title='Genre', yaxis_title='Total Sales')
genre_bar.show()

**Top Genres for each Region:**

| | NA           | EU           | JP           |
|-| :----------- | :----------- | :----------- |
|1| Action       | Action       | Role-Playing |
|2| Shooter      | Shooter      | Action       |
|3| Sports       | Sports       | Misc         |
|4| Role-Playing | Role-Playing | Fighting     |
|5| Misc         | Racing       | Shooter      |

- NA and EU markets look almost identical, with the exception of Racing taking the final spot in the EU region, possibly due to the prevalence of F1 Racing there.
- JP region still has a high prevalence of Action games but Role-Playing takes the first spot, likely due to JRPG titles flooding the market there. Fighting titles are also larger in this market.

### Rating

In [33]:
# Grouping by rating
df_rating = df_filtered.groupby('rating')[['na_sales', 'eu_sales', 'jp_sales']].sum().sort_values(by='na_sales')

# Graphing
rating_bar = px.bar(df_rating, barmode='group', title='ESRB Rating Sales by Region')
rating_bar.update_layout(xaxis_title='ESRB Rating', yaxis_title='Total Sales')
rating_bar.show()

Most of the entries in the JP region are unknown, potentially because the ESRB is a western company and would mostly rate games in the western market. All 3 markets are missing a significant portion of data.

| | NA       | EU       | JP       |
|-| :------- | :------- | :------- |
|1| **M**    | **M**    | **T**    |
|2| **E**    | **E**    | **E**    |
|3| **E10+** | **E10+** | **M**    |
|4| **T**    | **T**    | **E10+** |

- The **M** rating has a strong lead in the NA and EU regions. This can be attributed to the growing market being new and expensive hardware unavailable to younger demographics due to lack of funds, therefore most games in our filtered dataset are marketed towards an older audience.

- The **E** rating took second in every market, a nod to the idea that most games are marketed towards a younger audience.

- As the *PS4* and *XBox One* age, we can expect to see more games targeted at younger demograpics released for those platforms.

## Hypotheses Testing
***

### Testing whether average user ratings of the Xbox One and PC platforms are the same:

**Null Hypothesis:**

H<sub>0</sub>: μXbox = μPC

**Alternate Hypothesis:**

H<sub>1</sub>: μXbox ≠ μPC

- μPC = Population mean of user ratings for the PC platform
- μXbox = Population mean of user ratings for the Xbox One platform


The null hypothesis assumes the average user ratings for both XBox One and PC are the same, while the alternate hypothesis assumes they are different. The alpha value set for this test is 0.01%.

In [34]:
# Setting alpha value
alpha = 0.01

# Grouping data by platform (df_xone already exists)
df_pc = df_filtered[df_filtered['platform'] == 'PC']['user_score'].dropna()
df_xone_test = df_xone['user_score'].dropna()
# Preforming test
results = st.ttest_ind(df_xone_test, df_pc)

# Printing Results
print('p-value:', results.pvalue)

if (results.pvalue < alpha):
    print("We have sufficient evidence to reject the null hypothesis")
else:
    print("We do not have sufficient evidence to reject the null hypothesis")

p-value: 0.032907347893080446
We do not have sufficient evidence to reject the null hypothesis


There is not sufficient evidence to suggest that the average user rating is different between *XBox One* and *PC*.

### Testing whether average user ratings of the Action and Sports genres are different:

**Null Hypothesis:**

H<sub>0</sub>: μAction = μSports

- μAction = Population mean of user ratings for the Action Genre
- μSports = Population mean of user ratings for the Sports Genre

**Alternate Hypothesis:**

H<sub>1</sub>: μAction ≠ μSports

The null hypothesis assumes the average user ratings for both Action and Sports are the same, while the alternate hypothesis assumes they are different. The alpha value set for this test is 0.01%.

In [35]:
# Grouping data by genre
df_action = df_filtered[df_filtered['genre'] == 'Action']['user_score'].dropna()
df_sports = df_filtered[df_filtered['genre'] == 'Sports']['user_score'].dropna()

# Preforming test
results_genre = st.ttest_ind(df_action, df_sports)

# Printing Results
print('p-value:', results_genre.pvalue)

if (results_genre.pvalue < alpha):
    print("We have sufficient evidence to reject the null hypothesis")
else:
    print("We do not have sufficient evidence to reject the null hypothesis")

p-value: 2.7125375819769568e-34
We have sufficient evidence to reject the null hypothesis


There is enough evidence to suggest that the average user ratings differ between the Action and Sports genres.

## Conclusion
***

The analysis of the data has provided a few valuable insights into the data over the chosen period. The dataset as a whole represents an expanding industry showing significant growth, especially within the last few years. Notable all times sales leaders include the *PS2*, *XBox 360*, *PS3* and *Wii*. The newly released *PS4* and *XBox One* are leading in the current growth of the industry while most other platforms are declining in value.

The *PS3*, *PS4* and *3DS* all appear in the top 5 platforms for each region while *XBox* platforms are irrelevant to the JP region. Other consoles are more suited to global sales.

Further, while average user score had very little correlation with total sales, average critic score tended to have a positive correlation with sales, meaning a game with good critic reviews is more likely to sell better as opposed to games with good user reviews.

The entire industry tends toward the quicker-paced *Shooter*, *Sports* and *Platform* genres while customers in the JP region have a proclivity towards games of the Role-Playing genre. Most of the data tends away from slower paced *Puzzle*, *Strategy* and *Simulation* games.

While the data doesn't have sufficient evidence to suggest that ESRB ratings influence sales, a trend of more mature games being sold in the chosen time period was observed, possibly influenced by the recent release of the *XBox One* and *PS4*, both newer, more expensive hardware which is likely to have an older audience who can afford such expenses.

Overall, these observations underscore the dynamic nature of the video game industry, shaped by evolving consumer preferences, regional differences, and the interplay between ratings and sales.